In [1]:
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np

获取训练集中的物料名称向量

In [2]:
trainData = pd.read_csv("./Data_train.csv", index_col=0)
train_mat_names = list(trainData["MAT_NAME（物料名称）"].drop_duplicates())
trainData[:5]

,id（供应商代码）,MAT_NAME（物料名称）
QUOTE_MAT_LINE_ID（报价单行号）,,
8e24b75ce1c811ebaf97005056b12bb8,6603,液压缸吊座
f3afd8f4e1dc11ebaf97005056b12bb8,20958,六角头螺栓-C级GB/T5780-2000
1b99daf8e1de11ebaf97005056b12bb8,41982,高压水银荧光灯泡
1b9aac07e1de11ebaf97005056b12bb8,41982,手电筒螺口小灯泡（圆头）
bff9cf95e1e511ebaf97005056b12bb8,20238,外接头


In [3]:
trainMatVec = torch.load("trainMatVec"+".pt")

获取测试集中的物料名称向量

In [4]:
testData = pd.read_csv("./Data_train.csv", index_col = 0)
test_mat_names = list(testData["MAT_NAME（物料名称）"].drop_duplicates())

In [5]:
testData[:3]

,id（供应商代码）,MAT_NAME（物料名称）
QUOTE_MAT_LINE_ID（报价单行号）,,
8e24b75ce1c811ebaf97005056b12bb8,6603,液压缸吊座
f3afd8f4e1dc11ebaf97005056b12bb8,20958,六角头螺栓-C级GB/T5780-2000
1b99daf8e1de11ebaf97005056b12bb8,41982,高压水银荧光灯泡


计算测试集中物料名称与训练集中的物料名称的相似度

In [8]:
def cos_similar(p, q):
    sim_matrix = p.matmul(q.transpose(-2, -1))
    a = torch.norm(p, p=2, dim=-1)
    b = torch.norm(q, p=2, dim=-1)
    sim_matrix /= a.unsqueeze(-1)
    sim_matrix /= b.unsqueeze(-2)
    return sim_matrix
with torch.no_grad(): 
    # sim_mat_score = torch.cdist(trainMatVec,trainMatVec,p=2)
    sim_mat_score = cos_similar(trainMatVec,trainMatVec)
    sim_mat_score = sim_mat_score.sort(axis=1, descending=True)
    sim_mat_index = sim_mat_score.indices[:,:10]
    sim_mat_score = sim_mat_score.values[:,:10] 
torch.cuda.empty_cache()
print(sim_mat_score.shape)
print(sim_mat_index.shape)

torch.Size([31337, 10])
torch.Size([31337, 10])


In [9]:
sim_mat_index[:3]

tensor([[    0,   532, 16106, 17646,  9977,  9976,  3484, 17888,  4874, 16260],
        [    1,  6863, 20824,   847,   845, 30385,  2381,  2307,  6920,  9702],
        [    2, 22823, 15387, 23665, 23897, 15365,  5237, 15282, 27887, 28793]])

In [10]:
sim_mat_score[:3]

tensor([[1.0000, 0.9884, 0.9862, 0.9859, 0.9858, 0.9855, 0.9850, 0.9849, 0.9848,
         0.9847],
        [1.0000, 0.9986, 0.9806, 0.9800, 0.9798, 0.9795, 0.9795, 0.9793, 0.9789,
         0.9787],
        [1.0000, 0.9791, 0.9779, 0.9776, 0.9765, 0.9755, 0.9751, 0.9746, 0.9743,
         0.9735]])

In [11]:
train_mat_index = pd.DataFrame(np.arange(len(train_mat_names)), index = train_mat_names)
test_mat_index = pd.DataFrame(np.arange(len(test_mat_names)), index = test_mat_names)

In [12]:
train_mat_index.loc[test_mat_names[:3]]

,0
液压缸吊座,0
六角头螺栓-C级GB/T5780-2000,1
高压水银荧光灯泡,2


In [13]:
test_mat_index[:3]

,0
液压缸吊座,0
六角头螺栓-C级GB/T5780-2000,1
高压水银荧光灯泡,2


In [14]:
torch.save(sim_mat_index, "test-train-sim_mat_index-cosine"+".pt")
torch.save(sim_mat_score, "test-train-sim_mat_score-cosine"+".pt")

OSError: [Errno 122] Disk quota exceeded

In [6]:
sim_mat_index = torch.load("test-train-sim_mat_index"+".pt")
sim_mat_score = torch.load("test-train-sim_mat_score"+".pt")

获取历史每个物料对应的可提供供应商

In [15]:
train_id_mat = trainData.copy()
train_id_mat["mat_id"] = train_id_mat["MAT_NAME（物料名称）"].apply(lambda x:train_mat_index.loc[x])
train_id_mat[:2]

,id（供应商代码）,MAT_NAME（物料名称）,mat_id
QUOTE_MAT_LINE_ID（报价单行号）,,,
8e24b75ce1c811ebaf97005056b12bb8,6603,液压缸吊座,0
f3afd8f4e1dc11ebaf97005056b12bb8,20958,六角头螺栓-C级GB/T5780-2000,1


In [16]:
mat_supply_ids = train_id_mat.groupby(by="mat_id")["id（供应商代码）"].apply(set)
# 提供物料的历史供应商
mat_supply_ids[:200]

mat_id
0                                                 {6603}
1                                                {20958}
2                                                {41982}
3                                                {41982}
4                                  {27177, 21484, 20238}
                             ...                        
195                         {43944, 10298, 22123, 33944}
196                         {43944, 10298, 22123, 33944}
197                  {13094, 43944, 22123, 33944, 10298}
198                         {43944, 10298, 22123, 33944}
199    {43944, 22123, 24431, 32919, 33944, 13945, 10298}
Name: id（供应商代码）, Length: 200, dtype: object

In [17]:
query_mat_index = test_mat_index.loc[testData["MAT_NAME（物料名称）"].to_list()]
query_mat_index = query_mat_index[0].to_list()
# 物料在相似度矩阵中的下标
query_mat_index[:5]

[0, 1, 2, 3, 4]

In [18]:
item_sim_mat = pd.DataFrame(sim_mat_index[query_mat_index].tolist())
item_sim_mat[:4]

,0,1,2,3,4,5,6,7,8,9
0,0,532,16106,17646,9977,9976,3484,17888,4874,16260
1,1,6863,20824,847,845,30385,2381,2307,6920,9702
2,2,22823,15387,23665,23897,15365,5237,15282,27887,28793
3,3,1359,22734,15496,6350,7145,24149,23233,7144,15387


In [19]:
item_sim_score = pd.DataFrame(sim_mat_score[query_mat_index].tolist())
item_sim_score[:4]

,0,1,2,3,4,5,6,7,8,9
0,1.0,0.988395,0.986161,0.985858,0.985771,0.985544,0.985040,0.984938,0.984750,0.984743
1,1.0,0.998574,0.980628,0.979959,0.979800,0.979490,0.979490,0.979322,0.978859,0.978709
2,1.0,0.979115,0.977861,0.977573,0.976481,0.975545,0.975066,0.974640,0.974282,0.973462
3,1.0,0.977554,0.971322,0.970611,0.970597,0.970107,0.969902,0.969877,0.969649,0.969549


In [20]:
item_sim_mat.shape
# 行数是测试集数量，列数对应在训练集中相关的前10个物料向量

(220001, 10)

In [21]:
from operator import add
from functools import reduce
def simMat2supp(x, suppNum = 50):
    return reduce(add, mat_supply_ids.loc[x].apply(list))[:suppNum]
    # mat_supply_ids.loc[x]
    # return x
    # return mat_supply_ids.loc[x]
item_sim_mat[:10].apply(simMat2supp, axis=1)

0    [6603, 32877, 12702, 44683, 43181, 5262, 893, ...
1    [20958, 27208, 32114, 4591, 13237, 30944, 4297...
2    [41982, 525, 42240, 19906, 8840, 6478, 2415, 2...
3    [41982, 1125, 31877, 25865, 269, 5038, 32114, ...
4    [27177, 21484, 20238, 38648, 33250, 25356, 289...
5    [27177, 21484, 20238, 38648, 33250, 25356, 289...
6    [13585, 4381, 20238, 15234, 24710, 19272, 1684...
7    [8513, 12675, 7206, 29703, 43497, 20202, 41599...
8    [13585, 25972, 20238, 21484, 8513, 24067, 1267...
9    [9954, 3491, 35237, 13094, 40133, 21484, 20238...
dtype: object

In [22]:
result1 = item_sim_mat.apply(simMat2supp, axis=1)
result1 = pd.DataFrame(result1)
result1 = result1.set_index(testData.index)
result1.shape

(220001, 1)

In [23]:
testData["recommend"] = result1
testData[:3]

,id（供应商代码）,MAT_NAME（物料名称）,recommend
QUOTE_MAT_LINE_ID（报价单行号）,,,
8e24b75ce1c811ebaf97005056b12bb8,6603,液压缸吊座,"[6603, 32877, 12702, 44683, 43181, 5262, 893, ..."
f3afd8f4e1dc11ebaf97005056b12bb8,20958,六角头螺栓-C级GB/T5780-2000,"[20958, 27208, 32114, 4591, 13237, 30944, 4297..."
1b99daf8e1de11ebaf97005056b12bb8,41982,高压水银荧光灯泡,"[41982, 525, 42240, 19906, 8840, 6478, 2415, 2..."


In [24]:
sumRecall = testData.apply(lambda x:x["id（供应商代码）"] in x["recommend"], axis=1).sum()
recall = sumRecall / len(testData)
print(recall)

0.9564729251230676


In [23]:
testData.to_csv("trainDataRecommend"+".csv")

In [25]:
from operator import add
from functools import reduce
def simMat2supp(x, suppNum = 50):
    suplists = list(mat_supply_ids.loc[x].apply(list))
    scores = list(item_sim_score.loc[x.name])
    return pd.Series([suplists,scores])
item_sim_mat[:10].apply(simMat2supp, axis=1)

,0,1
0,"[[6603], [32877, 12702], [44683, 43181, 5262],...","[0.9999998211860657, 0.9883953332901001, 0.986..."
1,"[[20958], [27208, 32114, 4591], [13237], [3094...","[0.9999998807907104, 0.9985741972923279, 0.980..."
2,"[[41982], [525], [42240, 19906, 8840, 6478, 24...","[0.9999996423721313, 0.9791146516799927, 0.977..."
3,"[[41982], [1125, 31877, 25865, 269, 5038, 3211...","[1.0000004768371582, 0.9775543212890625, 0.971..."
4,"[[27177, 21484, 20238], [38648], [33250, 25356...","[1.000000238418579, 0.9894927740097046, 0.9879..."
5,"[[27177, 21484, 20238], [38648], [33250, 25356...","[1.000000238418579, 0.9894927740097046, 0.9879..."
6,"[[13585, 4381, 20238], [15234, 24710], [19272,...","[1.0000004768371582, 0.9849438667297363, 0.983..."
7,"[[8513, 12675, 7206, 29703, 43497, 20202, 4159...","[1.0, 1.0, 0.9882160425186157, 0.9873009920120..."
8,"[[13585, 25972, 20238, 21484], [8513, 24067, 1...","[1.000000238418579, 0.9962109923362732, 0.9906..."
9,"[[9954, 3491, 35237, 13094, 40133, 21484, 2023...","[1.0, 0.9923900961875916, 0.9910115599632263, ..."


In [26]:
result = item_sim_mat.apply(simMat2supp, axis=1)

In [27]:
result = result.rename(columns={0:'supps',1:'scores'})

In [28]:
result = result.set_index(testData.index)

In [29]:
result

,supps,scores
QUOTE_MAT_LINE_ID（报价单行号）,,
8e24b75ce1c811ebaf97005056b12bb8,"[[6603], [32877, 12702], [44683, 43181, 5262],...","[0.9999998211860657, 0.9883953332901001, 0.986..."
f3afd8f4e1dc11ebaf97005056b12bb8,"[[20958], [27208, 32114, 4591], [13237], [3094...","[0.9999998807907104, 0.9985741972923279, 0.980..."
1b99daf8e1de11ebaf97005056b12bb8,"[[41982], [525], [42240, 19906, 8840, 6478, 24...","[0.9999996423721313, 0.9791146516799927, 0.977..."
1b9aac07e1de11ebaf97005056b12bb8,"[[41982], [1125, 31877, 25865, 269, 5038, 3211...","[1.0000004768371582, 0.9775543212890625, 0.971..."
bff9cf95e1e511ebaf97005056b12bb8,"[[27177, 21484, 20238], [38648], [33250, 25356...","[1.000000238418579, 0.9894927740097046, 0.9879..."
...,...,...
d8d6af5cb70711ecbb62005056a00443,"[[39002], [41060, 40133, 40171, 1357, 9360, 46...","[1.0000003576278687, 0.9844587445259094, 0.984..."
d8d64183b70711ecbb62005056a00443,"[[39002], [43987, 14051], [17836, 20485], [178...","[1.0, 0.9757177829742432, 0.9733084440231323, ..."
d8d6b380b70711ecbb62005056a00443,"[[39002], [31480, 34758, 34270], [40339, 22724...","[1.0000004768371582, 0.9839216470718384, 0.983..."


In [26]:
pd.to_pickle(result, "trainDataRecommend.pkl")

In [198]:
result.to_csv("testDataRecommend.csv")

In [30]:
def scoreBroadCast(x, suppNum = 50):
    suppslist = []
    scoreslist = []
    for i in range(len(x['supps'])):
        suppslist.extend(x['supps'][i])
        scoreslist.extend([x['scores'][i]]*len(x['supps'][i]))
    return pd.Series([suppslist[:suppNum],scoreslist[:suppNum]])

In [31]:
broadResult = result.apply(scoreBroadCast, axis=1)

In [32]:
broadResult = broadResult.rename(columns={0:'supps',1:'scores'})
broadResult[:10]

,supps,scores
QUOTE_MAT_LINE_ID（报价单行号）,,
8e24b75ce1c811ebaf97005056b12bb8,"[6603, 32877, 12702, 44683, 43181, 5262, 893, ...","[0.9999998211860657, 0.9883953332901001, 0.988..."
f3afd8f4e1dc11ebaf97005056b12bb8,"[20958, 27208, 32114, 4591, 13237, 30944, 4297...","[0.9999998807907104, 0.9985741972923279, 0.998..."
1b99daf8e1de11ebaf97005056b12bb8,"[41982, 525, 42240, 19906, 8840, 6478, 2415, 2...","[0.9999996423721313, 0.9791146516799927, 0.977..."
1b9aac07e1de11ebaf97005056b12bb8,"[41982, 1125, 31877, 25865, 269, 5038, 32114, ...","[1.0000004768371582, 0.9775543212890625, 0.977..."
bff9cf95e1e511ebaf97005056b12bb8,"[27177, 21484, 20238, 38648, 33250, 25356, 289...","[1.000000238418579, 1.000000238418579, 1.00000..."
bffa5cd3e1e511ebaf97005056b12bb8,"[27177, 21484, 20238, 38648, 33250, 25356, 289...","[1.000000238418579, 1.000000238418579, 1.00000..."
bffa783fe1e511ebaf97005056b12bb8,"[13585, 4381, 20238, 15234, 24710, 19272, 1684...","[1.0000004768371582, 1.0000004768371582, 1.000..."
bffa926ae1e511ebaf97005056b12bb8,"[8513, 12675, 7206, 29703, 43497, 20202, 41599...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
bffa97c1e1e511ebaf97005056b12bb8,"[13585, 25972, 20238, 21484, 8513, 24067, 1267...","[1.000000238418579, 1.000000238418579, 1.00000..."


In [37]:
broadResult.to_csv("trainDataRecommendFormattedCosineSim.csv")

In [32]:
broadResult.to_pickle("trainDataRecommendFormatted.pkl")

In [124]:
lists = mat_supply_ids.loc[[197,198,199]].apply(list)

In [126]:
x = [197,198,199]
lists = mat_supply_ids.loc[x].apply(list)

In [167]:
lists

mat_id
197                  [13094, 43944, 22123, 33944, 10298]
198                         [43944, 10298, 22123, 33944]
199    [43944, 22123, 24431, 32919, 33944, 13945, 10298]
Name: id（供应商代码）, dtype: object

In [132]:
lists.apply(len).apply(lambda x:[1]) * lists.apply(len)

mat_id
197          [1, 1, 1, 1, 1]
198             [1, 1, 1, 1]
199    [1, 1, 1, 1, 1, 1, 1]
Name: id（供应商代码）, dtype: object

In [108]:
for i in range(10):
    

[6603]
[20958]
[41982]
